In [17]:
import pandas as pd
from pulp import *
import numpy as np
from scipy.optimize import minimize
import math
import time
from datetime import datetime

### Import Data
#### Demand per SKU 

In [ ]:
# Demand
df_demand = pd.read_csv('./data/df_demandsku.csv', index_col=0)
print("{:,} total demand".format(df_demand.DEMAND.sum()))
df_demand.head()

9,170 total demand


,SKU,DEMAND
0,D1,218
1,D2,277
2,D3,62
3,D4,142
4,D5,146


#### Cost per Carton

In [ ]:
df_costsku = pd.read_csv('./data/df_costsku.csv', index_col=0)
print("{:,} average cost per carton".format(df_costsku.COST.mean()))
df_costsku.head()

186.1 average cost per carton


,SKU,COST
0,D1,181
1,D2,126
2,D3,144
3,D4,238
4,D5,315


#### Transportation Costs

In [20]:
A = -0.3975
b = 42.250

### Build the model
#### Define the objective function

In [21]:
# Parameters for Transportation costs
A = -0.3975
b = 42.250

# Define the Objective Function
# Check APMonitor Youtube Channel for a great video tutorial https://www.youtube.com/watch?v=cXHvC_FGx24
def objective(R):
    result = 0
    for i in range(60):
        # TR Costs
        result += (A*(df_demand.loc[i,'DEMAND']/R[i]) + b) * R[i]
        # Capital Costs
        result += (df_demand.loc[i,'DEMAND']/(2*R[i])) * df_costsku.loc[i,'COST']*0.125
        # Storage Costs
        result += (df_demand.loc[i,'DEMAND']/(2*R[i])) * 12 * 480/2000
    return result 

### Add Constraints
#### Constraint 1: Maximum inventory

In [22]:
# Initialize constraints list
cons = []
# Maximum Inventory
def constraint1(R):
    loop = 0 
    for i in range(60):
        loop += R[i]
    result = 480 - loop
    return result
cons.append({'type':'ineq','fun':constraint1})

#### Constraint 2: Order Size

In [23]:
# Add Order Size Constraints
for i in range(60):
    # Minimum Order Quantity
    c2 = lambda R : (df_demand.loc[i,'DEMAND']/R[i]) - 1
    cons.append({'type':'ineq','fun':c2})
    # Maximum Order Quantity
    c3 = lambda R : 400 - (df_demand.loc[i,'DEMAND']/R[i]) 
    cons.append({'type':'ineq','fun':c3})

### Solve
#### Initial value (guessing)

In [24]:
# All SKU replenished 1 time
R0 = [2 for i in range(60)]
print("${:,} total cost for initial guessing".format(objective(R0).round(1)))

$63,206.7 total cost for initial guessing


#### Bounds

In [ ]:
# Bound vector
b_vector = (1, 365)
bnds = tuple([b_vector for i in range(60)])

---

### Solve with 100 iterations

In [ ]:
start = time.time()
sol = minimize(objective, R0, method = 'SLSQP', bounds=bnds, constraints = cons, options={'maxiter': 100})
exec_time = (time.time()-start)
print("Execution time is {}s for 100 iterations".format(exec_time))

#### Solution values

In [ ]:
# Initial solution
sol_init = sol.x
# Take the floor of the solution to have an integer as number of replenishment and never exceed stock limit 
sol_final = [math.floor(i) for i in sol_init]

#### Solution Results

In [ ]:
print(('''For {} Iterations
-> Initial Solution: ${:,}
-> Integer Solution: ${:,}
''').format(100, sol.fun.round(1), objective(sol_final).round(1)))

In [ ]:
print("Maximum inventory level with continuous number of replenishment: {}".format(sum(sol_init)))
print("Maximum inventory level with continuous number of replenishment: {}".format(sum(sol_final)))

---

### Solve with 500 iterations

In [ ]:
start = time.time()
sol = minimize(objective, R0, method = 'SLSQP', bounds=bnds, constraints = cons, options={'maxiter': 500})
exec_time = (time.time()-start)
print("Execution time is {}s for 500 iterations".format(exec_time))

#### Solution values

In [ ]:
# Initial solution
sol_init2 = sol.x
# Take the ceiling of the solution to have an integer as number of replenishment
sol_final2 = [math.ceil(i) for i in sol_init2]

#### Solution Results

In [ ]:
print(('''For {} Iterations
-> Initial Solution: ${:,}
-> Integer Solution: ${:,}
''').format(100, sol.fun.round(1), objective(sol_final2).round(1)))